In [96]:
urls = ['https://www.facebook.com/groups/950572615032243/']


In [97]:
import pandas as pd

collect_comment = pd.DataFrame()
collect_fb = pd.DataFrame() 

In [98]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [99]:
u = urls[0]

def FindLinks(url, n):
    Links = []
    driver.get(url)
    for i in range(n):
        time.sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        click_more(driver, '顯示先前')
        click_more(driver, '檢視另')
        click_more(driver, '查看另')
        click_more(driver, '已回覆')
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    for i in posts:
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
    return Links

In [100]:
from time import sleep

def click_more(driver, text):
    for i in range(3):
        comment_more = driver.find_elements_by_partial_link_text(text)
        if len(comment_more) == 0:
            break
        for btn in comment_more:
            try:
                btn.click()
                sleep(1)
                print('. ',end='')
            except:
                continue

In [101]:
!pip install lxml


In [102]:
import lxml

In [103]:
!pip install html5lib


In [104]:
import html5lib


In [105]:
import bs4


In [106]:
from bs4 import BeautifulSoup

In [107]:
htmltext = driver.page_source
soup = BeautifulSoup(htmltext)

MaxRetryError: HTTPConnectionPool(host='localhost', port=53676): Max retries exceeded with url: /session/aeb6e21905c478e2c2ef7781a009c4b8/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118f7fb80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
articles = soup.find_all('div', class_='_4-u2 mbm _4mrt _5jmm _5pat _5v3q _7cqq _4-u8') 

In [ ]:
def club_title(soup):
    return soup.title.string

In [ ]:
def article_person(article):
    return article.h5.a.string

In [ ]:
def article_date(article):
    return article.abbr.string

In [ ]:
def article_content(article):
    d = article_date(article)
    content = article.find('div', class_="_1dwg _1w_m _q7o").text.replace(d, '')
    return content

In [ ]:
def article_comment(article):
    try:
        comments = article.find('ul', class_="_7791").find_all('li')
        comment = []
        for i, c in enumerate(comments):
            if '隱藏或檢舉' in c.text:
                comment.append(c.text.split('隱藏或檢舉')[0])
            if '尋求支援' in c.text:
                comment.append(c.text.split('尋求支援')[0])
        return comment
    except:
        return []


In [ ]:
titles = []
sites = []
persons = []
dates = []
contents = []
comments = []
for a in articles:
    titles.append(club_title(soup))
    sites.append(u)
    persons.append(article_person(a))
    dates.append(article_date(a))
    contents.append(article_content(a))
    comments.append(article_comment(a))

In [ ]:
comments_len = [f'{len(c)}則留言' for c in comments]
df_fb = pd.DataFrame({'title': pd.Series(titles),
                      'site': pd.Series(sites),
                      'person': pd.Series(persons),
                      'date': pd.Series(dates),
                      'content': pd.Series(contents),
                      'comment': pd.Series(comments_len)
                     })
collect_fb = pd.concat([collect_fb, df_fb], ignore_index=True)
for c in comments:
    collect_comment = pd.concat([collect_comment, pd.Series(c)], axis=1)


In [ ]:
driver = webdriver.Chrome()
Links = FindLinks(url = u,
                  n = 20)
Links

In [ ]:
x, y = collect_comment.shape
collect_comment.columns = [f'index{i}' for i in range(y)]


In [ ]:
driver.quit()


In [ ]:
writer = pd.ExcelWriter('output2.xlsx', engine='xlsxwriter')
collect_fb.to_excel(writer, sheet_name='fb', encoding='utf-8')
collect_comment.to_excel(writer, sheet_name='comment', encoding='utf-8')
writer.save()